# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [ ]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 6.99MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import langchain
import langchain_groq
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import LLMChain
import langchain_core
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)



/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [4]:
df=pd.read_csv("Dataset.csv")
df

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
...,...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [6]:
import os
directory= 'data/markdownFile'

os.makedirs(directory, exist_ok=True)

In [11]:
for i in range(0,104):
  garama = df['الغرامة'].iloc[i]
  mkal = df['المخالفة'].iloc[i]

  markdown = f'** {mkal} - {garama}'


  with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
    file.write(markdown)



In [12]:
markdown_text= []
for filename in os.listdir(directory):
  if filename.endswith('.md'):
    with open(os.path.join(directory, filename) ,'r',encoding='utf-8') as file:
      markdown = file.read()
      markdown_text.append(markdown)



In [13]:
combine=markdown_text


In [14]:
combine

['** طمس لوحات المركبة. - الغرامة المالية 3000 - 6000 ريال',
 '** قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة المالية 100 - 150 ريال',
 '** عدم ربط حزام الأمان. - الغرامة المالية 150 - 300 ريال',
 '** استعمال المركبة لغير الغرض الذي رخصت من أجله. - الغرامة المالية 500 - 900 ريال',
 '** القيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها. - الغرامة المالية 1000 - 2000 ريال',
 '** عدم قيام السائق الذي يرغب في تغيير مساره بإعطاء الأفضلية لسائق المركبة التي تسير في اتجاه مستقيم في حال سير المركبتين متحاذيتين بشكل متواز. - الغرامة المالية 500 - 900 ريال',
 '** نقل الركاب في الأماكن غير المخصصة لهم في المركبة. - الغرامة المالية 500 - 900 ريال',
 '** قيام السائق بتخطي أرتال المركبات أمام إشارات المرور أو نقاط الضبط الأمني باستخدام كتف الطريق أو المسار المخصص للالتفاف. - الغرامة المالية 150 - 300 ريال',
 '** عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو الحافلات وما في حكمها في حال سيرها على المسارات المخصصة لها. - الغرامة المالية 500 - 900 ريا

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [15]:
split=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
data=split.create_documents(combine)

In [ ]:
data[0]

Document(metadata={}, page_content='** قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. الغرامة المالية 100 - 150 ريال')

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [26]:
#emb=SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
emb=SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v0")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.50k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [27]:
vectorstore=Chroma.from_documents(data,emb,persist_directory="./chroma_db_arabic2",)

In [28]:
vectorstore = Chroma(persist_directory="./chroma_db_arabic2",embedding_function=emb)

In [ ]:
vectorstore.similarity_search("مراوغة")

[Document(metadata={}, page_content='** العبث بعلامات الطريق أو العاكسات أو الشاحنات المنظمة لحركة السير. الغرامة المالية 3000 - 6000 ريال'),
 Document(metadata={}, page_content='** المجازفة بعبور الأودية والشعاب أثناء جريانها. الغرامة المالية 5000 - 10000 ريال'),
 Document(metadata={}, page_content='** المراوغة بسرعة بين المركبات على الطرق العامة. الغرامة المالية 3000 - 6000 ريال'),
 Document(metadata={}, page_content='** وضع كتابة أو رسم أو ملصق أو أي بيان آخر على جسم المركبة دون موافقة الجهات المختصة. الغرامة المالية 500 - 900 ريال')]

In [19]:
vectorstore.similarity_search("مراوغة")

[Document(metadata={}, page_content='** التجمهر في مواقع التفحيط. - الغرامة المالية 1000 - 2000 ريال'),
 Document(metadata={}, page_content='** طمس رقم هيكل المركبة أو محاولة طمسه. - الغرامة المالية 5000 - 10000 ريال'),
 Document(metadata={}, page_content='** سير المركبة بلا لوحة خلفية، أو بلا لوحات. - الغرامة المالية 3000 - 6000 ريال'),
 Document(metadata={}, page_content='** طمس لوحات المركبة. - الغرامة المالية 3000 - 6000 ريال')]

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [20]:
test="كم مخالفة المراوغة"
con=vectorstore.similarity_search(f"{test}",k=9)
contex=""
for i in con:
  contex+=("\n"+i.page_content)

print(contex)


** التجمهر في مواقع التفحيط. - الغرامة المالية 1000 - 2000 ريال
** استعمال المكابح (الفرامل) بشكل مفاجئ لغير ضرورة. - الغرامة المالية 300 - 500 ريال
** التجاوز في المناطق التي يمنع فيها التجاوز، مثل المنعطفات والمرتفعات. - الغرامة المالية 1000 - 2000 ريال
** مخالفة قواعد استعمال أنوار التلاقي. - الغرامة المالية 150 - 300 ريال
** طمس رقم هيكل المركبة أو محاولة طمسه. - الغرامة المالية 5000 - 10000 ريال
** المجازفة بعبور الأودية والشعاب أثناء جريانها. - الغرامة المالية 5000 - 10000 ريال
** عدم إنهاء إجراءات تعديل مجال استعمال المركبة. - الغرامة المالية 1000 - 2000 ريال
** طمس لوحات المركبة. - الغرامة المالية 3000 - 6000 ريال
** عدم الوقوف تماماً عند إشارة قف. - الغرامة المالية 500 - 900 ريال


In [35]:
context_for_model="""
 بناء على المعلومات التالية فقط اجب على السؤال التالي  ويرجى اعطاء نصيحة الاجابة فقط بالعربي
السياق : {con}
السؤال : {qu}
اجابتك:
"""
prompt_template = langchain_core.prompts.prompt.PromptTemplate(
    template=context_for_model, input_variables=["con", "qu"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [30]:
llm=langchain_groq.ChatGroq(model="llama3-70b-8192",temperature=0.7,api_key="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")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [36]:
chain=langchain.chains.LLMChain(llm=llm,prompt=prompt_template)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [37]:
def rag_fun(text):
  con=vectorstore.similarity_search(f"{text}",k=6)

  contex=""
  for i in con:
    contex+=(i.page_content+"\n")
  return contex

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [38]:
text_user=input("Enter your q:")
con=rag_fun(text_user)
con
out=chain.invoke({"con":con,"qu":text_user})
print(out["text"])

Enter your q:ماهي الغرامة على القيادة بدون رخصة؟
الغرامة المالية 150 - 300 ريال.
